# SFC版DQ3 呪文習得について

In [2]:
:dep dq3 = { path = "../" }
:dep polars = "0.21.1"
:dep plotly = "0.7.0"
use dq3;
use polars::prelude::*;
use polars::df;
use plotly;

このNotebookでは、SFC版ドラゴンクエスト3(DQ3)における呪文習得の仕様について述べる。

## 1. 呪文習得に関係するパラメータ
呪文習得に大きく関係するデータ構造として、下記の2つが存在する。
これらの実際のデータ値に関しては[スプレッドシート](https://docs.google.com/spreadsheets/d/1KzoLM5tUFkL0oLLEicw4bK3Ki8RktnXAaT5-pBe8MWE/edit?usp=sharing) を参照してほしい。

### 1.1. 習得レベル
職業ごとに各呪文に対する習得レベルが設定されている。習得レベルとは、その職業が当該呪文を習得するために必要な最低レベルのことである。

### 1.2. 習得タイプ
呪文ごとに習得タイプが設定されている。習得タイプは2種類存在し、0 または 1の値として表現される。

それぞれの習得タイプの詳細については次節にて説明する。

## 2. 呪文習得タイプ
### 2.1. 習得タイプ0: かしこさ参照習得
タイプ0の呪文は、キャラクターのレベルとかしこさを参照して呪文の習得判定が行われるものである。
RTAでよく用いられる呪文の中では、 `ヒャド` 、 `ベホイミ` 、 `ヒャダルコ` 、 `ザオラル` などが取得タイプ0に該当する。

#### 2.1.1. かしこさ期待値
タイプ0の呪文習得では、下記の式で定義される `かしこさ期待値` を利用して習得判定が行われる。

$$ かしこさ期待値(職業, レベル) = + \sum_{n=2}^{レベル-1}標準成長値(職業, n, かしこさ) + 5 $$

即ち、 __Lv2からLvUP後のLv-1までのかしこさ標準成長値を合計した値に5を足した値__ である。標準成長値についてはステータス成長についての文章で説明しているので、そちらを参照して欲しい。

ステータス成長における `ステータス標準値` と計算方法が似ているが、 `初期値` を合算しないこと、 レベル-1 までの標準成長値の合計であることの2点が異なる。

またここから、__かしこさ期待値は職業によって異なる__ という特徴が分かる。例えば __僧侶及び魔法使いと賢者は同じ呪文を同じ習得レベルで習得するが、それぞれの職業の間で参照されるかしこさ期待値は異なる__ ということである。

#### 2.1.2. 習得タイプ0の呪文習得判定フロー
習得タイプ0の呪文の習得判定処理は下記のようなフローになっている。

1. $LvUP後のかしこさ - かしこさ期待値 ≥ 11$ の場合:
   1. $Lv ≥ 習得Lv$であれば習得
2. $11 > LvUP後のかしこさ - かしこさ期待値 ≥ -15$ の場合:
   1. 0 or 1 の乱数を取得 ([0, 256) の一様整数乱数を2で割った剰余で計算される) 
      1. 乱数=0 => $Lv ≥ 習得Lv$   であれば習得
      2. 乱数=1 => $Lv ≥ 習得Lv+1$ であれば習得
3. $-15 ≥ LvUP後のかしこさ - かしこさ期待値$ の場合:
   1. 0 or 1 の乱数を取得 ([0, 256) の一様整数乱数を2で割った剰余で計算される)
      1. 乱数=0 => $Lv ≥ 習得Lv+1$ であれば習得
      2. 乱数=1 => $Lv ≥ 習得Lv+2$ であれば習得

わかりやすく表形式でまとめると下記のようになる。

<table dir="ltr">
    <th align="left">diff := かしこさ - かしこさ期待値</th>
    <td align="center">diff ≥ 11</td>
    <td align="center" colspan="2">11 > diff ≥ -15</td>
    <td align="center" colspan="2">-15 > diff</td>
  </tr>
  <tr>
    <th align="left">rand % 2</th>
    <td align="center">N/A</td>
    <td align="center">1</td>
    <td align="center">0</td>
    <td align="center">1</td>
    <td align="center">0</td>
  </tr>
  <tr>
    <th align="left">Lv</th>
    <td align="center">Lv ≥ 習得Lv</td>
    <td align="center">Lv ≥ 習得Lv</td>
    <td align="center">Lv ≥ 習得Lv+1</td>
    <td align="center">Lv ≥ 習得Lv+1</td>
    <td align="center">Lv ≥ 習得Lv+2</td>
  </tr>
</table>

即ち、 __習得タイプ0の呪文はかしこさが一定の値以上であれば確定で習得をすることが出来る。また最悪の場合でも、習得Lv+2 までのレベルには必ず習得出来る。__

### 2.2. 習得タイプ1: 完全ランダム習得
タイプ1の呪文は、__キャラクターのレベルが習得レベル以上のとき、1/2の確率で呪文を習得する。__
RTAでよく用いられる呪文の中では、 `スカラ` 、 `ギラ` 、 `バイキルト` などが取得タイプ0に該当する。

習得タイプ0の習得判定処理は下記のようなフローになっている。

1. 0 or 1 の乱数を取得 ([0, 256) の一様整数乱数を2で割った剰余で計算される) 
   1. 乱数=0 => 習得しない
   2. 乱数=1 => $Lv ≥ 習得Lv$ であれば習得

タイプ0と違い、確定習得レベルが存在しないことが特徴である。最悪の場合では、Lv99まで習得しない可能性もある。

## 3. RTAにおいて重要となる呪文の確定習得に必要なかしこさ

In [3]:
println!("けんじゃ Lv14 ベホイミ: {}", dq3::job::get_job_table(dq3::job::Job::Sage).intelligence_thresh_for_learning(14).1);
println!("けんじゃ Lv24 ザオラル: {}", dq3::job::get_job_table(dq3::job::Job::Sage).intelligence_thresh_for_learning(24).1);
println!("けんじゃ Lv32 フバーハ: {}", dq3::job::get_job_table(dq3::job::Job::Sage).intelligence_thresh_for_learning(32).1);
println!("まほうつかい Lv36 メラゾーマ: {}", dq3::job::get_job_table(dq3::job::Job::Wizard).intelligence_thresh_for_learning(36).1);

けんじゃ Lv14 ベホイミ: 28
けんじゃ Lv24 ザオラル: 45
けんじゃ Lv32 フバーハ: 68
まほうつかい Lv36 メラゾーマ: 102


## 4. かしこさ期待値の図示
各職業の各レベルにおけるかしこさ期待値を図示してみる。

In [12]:
fn learning_int_thresh_dataframe(job: dq3::job::Job) -> DataFrame {
    let attr = dq3::attr::Attr::Int;
    let job_table = dq3::job::get_job_table(job);

    let lvs: Vec<u8> = (2..100).collect();    
    let incrs: Series = lvs.iter().map(|&lv| job_table.growth_value(lv, attr).to_num::<f64>()).collect();    
    let stds: Series = lvs.iter().map(|&lv| job_table.standard_intelligence_by_learning(lv) as u32).collect();
    let threshs: Vec<_> = lvs.iter().map(|&lv| job_table.intelligence_thresh_for_learning(lv)).collect();
    let threshs0: Series = threshs.iter().map(|thresh| thresh.0 as u32).collect();
    let threshs1: Series = threshs.iter().map(|thresh| thresh.1 as u32).collect();

    let lvs: Series = (2..100u32).collect();

    std::env::set_var("POLARS_FMT_MAX_ROWS", "100");

    let status_df = df!(
        "Lv" => &lvs,
        "かしこさ標準成長値" => &incrs,
        "かしこさ期待値" => &stds,
        "かしこさ期待値-15" => &threshs0,
        "かしこさ期待値+11" => &threshs1
        );
    
    status_df.unwrap()
}

### 4.1. まほうつかい

In [13]:
learning_int_thresh_dataframe(dq3::job::Job::Wizard)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 2.0                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 2.0                ┆ 7              ┆ 0                 ┆ 18                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 2.0                ┆ 9              ┆ 0                 ┆ 20                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 2.0                ┆ 11             ┆ 0                 ┆ 22 

### 4.2. そうりょ

In [14]:
learning_int_thresh_dataframe(dq3::job::Job::Pligrim)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 1.75               ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 1.75               ┆ 6              ┆ 0                 ┆ 17                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 1.75               ┆ 8              ┆ 0                 ┆ 19                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 1.75               ┆ 10             ┆ 0                 ┆ 21 

### 4.3. しょうにん

In [15]:
learning_int_thresh_dataframe(dq3::job::Job::Merchant)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 1.0                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 1.0                ┆ 6              ┆ 0                 ┆ 17                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 1.0                ┆ 7              ┆ 0                 ┆ 18                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 1.0                ┆ 8              ┆ 0                 ┆ 19 

### 4.4. あそびにん

In [16]:
learning_int_thresh_dataframe(dq3::job::Job::GoofOff)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 3.0                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 3.0                ┆ 8              ┆ 0                 ┆ 19                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 3.0                ┆ 11             ┆ 0                 ┆ 22                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 3.0                ┆ 14             ┆ 0                 ┆ 25 

### 4.5. とうぞく

In [17]:
learning_int_thresh_dataframe(dq3::job::Job::Thief)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 2.0                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 2.0                ┆ 7              ┆ 0                 ┆ 18                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 2.0                ┆ 9              ┆ 0                 ┆ 20                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 2.0                ┆ 11             ┆ 0                 ┆ 22 

### 4.6. けんじゃ

In [18]:
learning_int_thresh_dataframe(dq3::job::Job::Sage)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 0.5                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 0.5                ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 0.5                ┆ 6              ┆ 0                 ┆ 17                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 0.5                ┆ 6              ┆ 0                 ┆ 17 

### 4.7. ゆうしゃ

In [19]:
learning_int_thresh_dataframe(dq3::job::Job::Hero)

shape: (98, 5)
┌─────┬────────────────────┬────────────────┬───────────────────┬───────────────────┐
│ Lv  ┆ かしこさ標準成長値 ┆ かしこさ期待値 ┆ かしこさ期待値-15 ┆ かしこさ期待値+11 │
│ --- ┆ ---                ┆ ---            ┆ ---               ┆ ---               │
│ u32 ┆ f64                ┆ u32            ┆ u32               ┆ u32               │
╞═════╪════════════════════╪════════════════╪═══════════════════╪═══════════════════╡
│ 2   ┆ 0.75               ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3   ┆ 0.75               ┆ 5              ┆ 0                 ┆ 16                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4   ┆ 0.75               ┆ 6              ┆ 0                 ┆ 17                │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5   ┆ 0.75               ┆ 7              ┆ 0                 ┆ 18 